In [3]:
import pandas as pd
from kmodes.kprototypes import KPrototypes
from sklearn.preprocessing import StandardScaler

In [28]:
df = pd.read_csv("enviroTab_pa_train.csv", sep=';')
df_sans_na = df.dropna()
df_sans_na

,patchID,dayOfYear,lon,lat,x_EPSG3035,y_EPSG3035,year,dataset,geoUncertaintyInM,bio1,...,Built1994,Lights1994,NavWater1994,Built2009,Lights2009,NavWater2009,Popdensity1990,Popdensity2010,Railways,Roads
0,55880,126,4.74408,44.20370,3.900369e+06,2.358486e+06,2020,CBNMed,10.0,2878,...,0.0,7.0,0.357577,10.0,8.0,0.357577,6.0,6.0,0.0,1.629548
1,55897,126,4.74436,44.20410,3.900395e+06,2.358529e+06,2020,CBNMed,10.0,2878,...,0.0,7.0,0.357577,10.0,8.0,0.357577,6.0,6.0,0.0,1.629548
2,55942,126,4.74519,44.20360,3.900457e+06,2.358469e+06,2020,CBNMed,10.0,2878,...,0.0,7.0,0.357577,10.0,8.0,0.357577,6.0,6.0,0.0,1.629548
3,56533,126,4.75466,44.21930,3.901329e+06,2.360153e+06,2020,CBNMed,10.0,2876,...,0.0,4.0,0.144801,0.0,5.0,0.144801,6.0,6.0,0.0,0.852498
4,56759,126,4.75755,44.21360,3.901518e+06,2.359506e+06,2020,CBNMed,10.0,2877,...,0.0,4.0,0.144801,0.0,5.0,0.144801,6.0,6.0,0.0,0.852498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5943,3018845,138,9.28662,41.87388,4.261560e+06,2.085921e+06,2018,Inventaire forestier IGN,1.0,2839,...,0.0,0.0,0.002858,0.0,0.0,0.002858,2.0,2.0,0.0,0.757507
5944,3018847,351,9.29834,41.56691,4.262244e+06,2.051949e+06,2018,Inventaire forestier IGN,1.0,2898,...,0.0,3.0,1.281452,0.0,6.0,1.281452,5.0,6.0,0.0,1.629548
5945,3018848,346,9.31529,41.70986,4.263796e+06,2.067751e+06,2017,Inventaire forestier IGN,1.0,2894,...,0.0,1.0,0.042134,0.0,1.0,0.042134,4.0,4.0,0.0,4.000000
5946,3018853,346,9.35871,41.67093,4.267389e+06,2.063412e+06,2018,Inventaire forestier IGN,1.0,2897,...,0.0,3.0,1.788531,0.0,4.0,1.788531,3.0,4.0,0.0,0.526984


In [36]:
quanti = df_sans_na.iloc[:,9:37]
variables_quantitatives = quanti.apply(lambda col: col.fillna(col.mean()), axis=0)

In [8]:
df.columns

Index(['patchID', 'dayOfYear', 'lon', 'lat', 'x_EPSG3035', 'y_EPSG3035',
       'year', 'dataset', 'geoUncertaintyInM', 'bio1', 'bio2', 'bio3', 'bio4',
       'bio5', 'bio6', 'bio7', 'bio8', 'bio9', 'bio10', 'bio11', 'bio12',
       'bio13', 'bio14', 'bio15', 'bio16', 'bio17', 'bio18', 'bio19', 'bdod',
       'cec', 'cfvo', 'clay', 'nitrogen', 'phh2o', 'sand', 'silt', 'soc',
       'landCov', 'Built1994', 'Lights1994', 'NavWater1994', 'Built2009',
       'Lights2009', 'NavWater2009', 'Popdensity1990', 'Popdensity2010',
       'Railways', 'Roads'],
      dtype='object')

In [37]:
quali = df_sans_na[['landCov','Popdensity2010', 'Built2009', 'Lights2009', 'Railways', 'Roads']]
variables_qualitatives = quali.apply(lambda col: col.fillna(col.mode()[0]), axis=0)

In [38]:
scaler = StandardScaler()
variables_quantitatives_standardisees = scaler.fit_transform(variables_quantitatives)

# Concaténation des variables quantitatives standardisées et des variables catégorielles
X = pd.concat([pd.DataFrame(variables_quantitatives_standardisees, columns=variables_quantitatives.columns), variables_qualitatives], axis=1)

# Spécification du nombre de clusters (à ajuster)
nombre_clusters = 3

# Application de l'algorithme K-Prototypes
kproto = KPrototypes(n_clusters=nombre_clusters, init='Cao', verbose=2)
clusters = kproto.fit_predict(X, categorical=[X.columns.get_loc(col) for col in variables_qualitatives.columns])

# Ajout des étiquettes de cluster au DataFrame d'origine
df['cluster'] = clusters

# Visualisation des clusters (à adapter selon vos variables)
# Par exemple, une visualisation en 2D pour deux variables quantitatives
plt.scatter(df['variable1'], df['variable2'], c=df['cluster'], cmap='viridis')
plt.title('Clusters')
plt.xlabel('Variable 1')
plt.ylabel('Variable 2')
plt.show()

Initialization method and algorithm are deterministic. Setting n_init to 1.


ValueError: Input contains NaN.